In [199]:
from __future__ import print_function
import argparse
import torch
import torch.utils.data
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [200]:
parser = argparse.ArgumentParser(description='PyTorch MNIST Example')
parser.add_argument('--batch-size', type=int, default=64, metavar='N',
                    help='input batch size for training (default: 128)')
parser.add_argument('--epochs', type=int, default=10, metavar='N',
                    help='number of epochs to train (default: 10)')
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
parser.add_argument("--n_gpu", type=int, default=1, help="number of gpu")
parser.add_argument("--latent_dim", type=int, default=32, help="dimensionality of the latent space")
parser.add_argument("--ngf", type=int, default=64, help="numbers of generator's features")
parser.add_argument("--ndf", type=int, default=64, help="numbers of discriminator's features")
parser.add_argument("--img_size", type=int, default=28, help="size of each image dimension")
parser.add_argument("--channels", type=int, default=1, help="number of image channels")
parser.add_argument('--seed', type=int, default=1, metavar='S',
                    help='random seed (default: 1)')
parser.add_argument('--log-interval', type=int, default=10, metavar='N',
                    help='how many batches to wait before logging training status')
args = parser.parse_known_args()[0]
args.cuda = True if torch.cuda.is_available() else False
print(args)

#Sets the seed for generating random numbers. And returns a torch._C.Generator object.
#torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)     

Namespace(batch_size=64, channels=1, cuda=True, epochs=10, img_size=28, latent_dim=32, log_interval=10, n_cpu=8, n_gpu=1, ndf=64, ngf=64, seed=1)


In [201]:
kwargs = {'num_workers': 1, 'pin_memory': True} if args.cuda else {}
trainset = datasets.MNIST('../data', train=True, download=True,transform=transforms.ToTensor())
train_loader = torch.utils.data.DataLoader(
    trainset,
    batch_size=args.batch_size, shuffle=True, **kwargs)
image, label = trainset[0]  
print(len(trainset))
print(image.size())

60000
torch.Size([1, 28, 28])


In [203]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, args.latent_dim)
        self.fc22 = nn.Linear(400, args.latent_dim)
        
        self.fc3 = nn.Linear(args.latent_dim, 400)
        self.fc4 = nn.Linear(400, 784)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        eps = Variable(std.data.new(std.size()).normal_())
        return eps.mul(std).add_(mu)

    def decoder(self, z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparametrize(mu, logvar)
        return self.decoder(z), mu, logvar

vae = VAE()
if args.cuda:
    vae.cuda()

In [204]:
reconstruction_function = nn.BCELoss()
reconstruction_function.size_average = False

def loss_function(recon_x, x, mu, logvar):
    #BCE = reconstruction_function(recon_x, x.view(-1, 784))
    BCE = F.binary_cross_entropy(recon_x, x,  size_average=False)

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.sum(KLD_element).mul_(-0.5)

    return BCE + KLD


#optimizer = optim.Adam(model.parameters(), lr=0.1)
optimizer =  optim.Adam(vae.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0)

In [205]:
def train():
    vae.train()
    total_loss = 0
    for i, (data, _) in enumerate(train_loader, 0):
        data = Variable(data).cuda(0)
        optimizer.zero_grad()
        recon_x, mu, logvar = vae.forward(data)
        loss = loss_function(recon_x, data, mu, logvar)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
        
        if i % args.log_interval == 0:
            sample = Variable(torch.randn(64, args.latent_dim)).cuda()
            #print(sample)
            sample = vae.decoder(sample).cpu()
            save_image(sample.data.view(64, 1, 28, 28),
                       'result_Linear/' + str(epoch) + '.png')
            print('Train Epoch:{} -- [{}/{} ({:.0f}%)] -- Loss:{:.6f}'.format(
                epoch, i*len(data), len(train_loader.dataset), 
                100.*i/len(train_loader), loss.item()/len(data)))
            
#         print('====> Epoch: {} Average loss: {:.4f}'.format(
#             epoch, total_loss / len(train_loader.dataset)))

In [206]:
for epoch in range(1, args.epochs + 1):
    train()

C:\Users\ShawnD98\Anaconda3\envs\Deeplearning\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Using a target size (torch.Size([64, 1, 28, 28])) that is different to the input size (torch.Size([64, 784])) is deprecated. Please ensure they have the same size.
  


Train Epoch:1 -- [0/60000 (0%)] -- Loss:546.180420
Train Epoch:1 -- [640/60000 (1%)] -- Loss:292.660736
Train Epoch:1 -- [1280/60000 (2%)] -- Loss:240.823273
Train Epoch:1 -- [1920/60000 (3%)] -- Loss:218.934601
Train Epoch:1 -- [2560/60000 (4%)] -- Loss:208.608902
Train Epoch:1 -- [3200/60000 (5%)] -- Loss:214.375336
Train Epoch:1 -- [3840/60000 (6%)] -- Loss:207.731201
Train Epoch:1 -- [4480/60000 (7%)] -- Loss:211.728882
Train Epoch:1 -- [5120/60000 (9%)] -- Loss:195.699158
Train Epoch:1 -- [5760/60000 (10%)] -- Loss:189.079681
Train Epoch:1 -- [6400/60000 (11%)] -- Loss:186.994278
Train Epoch:1 -- [7040/60000 (12%)] -- Loss:185.026382
Train Epoch:1 -- [7680/60000 (13%)] -- Loss:171.197739
Train Epoch:1 -- [8320/60000 (14%)] -- Loss:164.202377
Train Epoch:1 -- [8960/60000 (15%)] -- Loss:175.506348
Train Epoch:1 -- [9600/60000 (16%)] -- Loss:174.201355
Train Epoch:1 -- [10240/60000 (17%)] -- Loss:158.925552
Train Epoch:1 -- [10880/60000 (18%)] -- Loss:164.889709
Train Epoch:1 -- [115

C:\Users\ShawnD98\Anaconda3\envs\Deeplearning\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Using a target size (torch.Size([32, 1, 28, 28])) that is different to the input size (torch.Size([32, 784])) is deprecated. Please ensure they have the same size.
  


Train Epoch:2 -- [0/60000 (0%)] -- Loss:126.617615
Train Epoch:2 -- [640/60000 (1%)] -- Loss:123.485306
Train Epoch:2 -- [1280/60000 (2%)] -- Loss:125.762917
Train Epoch:2 -- [1920/60000 (3%)] -- Loss:122.089294
Train Epoch:2 -- [2560/60000 (4%)] -- Loss:124.562263
Train Epoch:2 -- [3200/60000 (5%)] -- Loss:126.500397
Train Epoch:2 -- [3840/60000 (6%)] -- Loss:124.491592
Train Epoch:2 -- [4480/60000 (7%)] -- Loss:119.583427
Train Epoch:2 -- [5120/60000 (9%)] -- Loss:130.571503
Train Epoch:2 -- [5760/60000 (10%)] -- Loss:120.408760
Train Epoch:2 -- [6400/60000 (11%)] -- Loss:125.666260
Train Epoch:2 -- [7040/60000 (12%)] -- Loss:119.181305
Train Epoch:2 -- [7680/60000 (13%)] -- Loss:120.804581
Train Epoch:2 -- [8320/60000 (14%)] -- Loss:121.163742
Train Epoch:2 -- [8960/60000 (15%)] -- Loss:121.209656
Train Epoch:2 -- [9600/60000 (16%)] -- Loss:119.376923
Train Epoch:2 -- [10240/60000 (17%)] -- Loss:111.205070
Train Epoch:2 -- [10880/60000 (18%)] -- Loss:117.772461
Train Epoch:2 -- [115